In [1]:
%matplotlib inline

import cv2
import itertools as it
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import random
from collections import Counter
from utilities import *

def show_square(images, size=30):
    g = (cv2.resize(image, (size, size)) for image in images)
    n = math.ceil(math.sqrt(len(images)))
    z = np.zeros_like(images[0])[:size, :size]
    g = (np.hstack(list(g)) for g in grouper(g, n, fillvalue=z))
    return show_and_wait(np.vstack(list(g)))

In [2]:
images = list(pickle_iter(r"D:\Valorant\Killed\Color\00001.pickle"))
len(images)

8000

In [3]:
for i, image in enumerate(images):
    if show_and_wait(image, wait_time=1) == 'q':
        print(i)
        break
cv2.destroyAllWindows()

In [4]:
upper_positives, lower_positives = images[6096:6296], images[7748:7998]
positives = upper_positives + lower_positives
def fn(image):
    return cv2.resize(image, (30, 30))
g = grouper(positives, 25)
g = (np.hstack(list(map(fn, g))) for g in g)
show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()
list(prime_factors(len(positives)))

[2, 3, 3, 5, 5]

In [5]:
for i, image in enumerate(positives):
    if show_and_wait(image, wait_time=25) == 'q':
        break
cv2.destroyAllWindows()

In [6]:
left, right = 3, 59
width, height = right - left, 11
upper_extent = (59, left, 59 + height, right)
lower_extent = (95, left, 95 + height, right)
g = ((upper_positives, upper_extent), (lower_positives, lower_extent))
g = (zip(a, it.repeat(b)) for a, b in g)
g = it.chain.from_iterable(g)
g = (i[top:bottom, left:right] for i, (top, left, bottom, right) in g)
g = grouper(g, 10)
g = (np.hstack(list(g)) for g in g)
show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()
(upper_extent[2] - upper_extent[0], upper_extent[3] - upper_extent[1],
 lower_extent[2] - lower_extent[0], lower_extent[3] - lower_extent[1])

(11, 56, 11, 56)

In [7]:
g = ((upper_positives, upper_extent), (lower_positives, lower_extent))
g = (zip(a, it.repeat(b)) for a, b in g)
g = it.chain.from_iterable(g)
g = (i[top:bottom, left:right] for i, (top, left, bottom, right) in g)
color = np.stack(list(g)).mean(axis=(0, 1, 2))
show_and_wait(np.broadcast_to(color, (222, 222, 3)))
cv2.destroyAllWindows()
color.shape, color

((3,), array([0.5691015, 0.6987058, 0.7345293], dtype=float32))

In [8]:
template = np.broadcast_to(color, (height, width, 3))
match = cv2.matchTemplate(upper_positives[0][:, left:right], template, cv2.TM_SQDIFF)
show_and_wait(np.broadcast_to(match / match.max(), (match.shape[0], 222)))
cv2.destroyAllWindows()
match.reshape(-1), match.min(), match.max()

(array([128.6426  , 127.5322  , 126.27965 , 124.479324, 122.508446,
        120.03372 , 117.15281 , 113.81479 , 110.16839 , 106.23877 ,
        102.02189 ,  97.77    ,  93.92924 ,  90.34311 ,  87.257164,
         84.179245,  81.32891 ,  78.797295,  76.677055,  74.83733 ,
         73.34904 ,  72.17568 ,  71.07899 ,  69.86489 ,  68.81485 ,
         67.67303 ,  66.589264,  65.36002 ,  64.25471 ,  63.069504,
         61.682972,  60.015312,  58.082607,  56.29231 ,  54.69686 ,
         53.05775 ,  51.70887 ,  50.49536 ,  49.435486,  48.319233,
         47.3587  ,  46.602898,  46.04036 ,  45.617027,  45.502365,
         45.366463,  44.49129 ,  41.370388,  38.154526,  35.01607 ,
         31.871313,  28.760847,  25.70094 ,  22.658968,  19.793331,
         16.611198,  13.406542,  11.025362,  10.860258,  10.809429,
         11.071693,  12.512628,  31.637377,  51.375805,  71.55317 ,
         91.3508  , 109.59195 , 127.389824, 146.61267 , 164.92969 ,
        181.7354  , 207.4926  , 228.01678 , 230.

In [9]:
negatives = random.sample(range(6000), 340) + random.sample(range(6310, 7745), 110)
negatives = [images[i] for i in sorted(negatives)]
g = (np.hstack([cv2.resize(image, (55, 55)) for image in g]) for g in grouper(negatives, 25))
show_and_wait(np.vstack(list(g)))
cv2.destroyAllWindows()

In [10]:
len(positives), positives[0].shape, len(negatives), negatives[0].shape

(450, (108, 168, 3), 450, (108, 168, 3))

In [11]:
def fn(image):
    match = cv2.matchTemplate(image[:, left:right], template, cv2.TM_SQDIFF)
    return match
image = np.hstack(list(map(fn, negatives + positives)))
show_and_wait(image / image.max())
cv2.destroyAllWindows()

In [12]:
def fn(image):
    # Using RGB and match.min() < 42 results in
    # [Counter({False: 360, True: 90}), Counter({True: 448, False: 2})]
    n, d = 25, math.log(2)
    image = cv2.cvtColor(image[:, left:right], cv2.COLOR_BGR2LUV)
    match = cv2.matchTemplate(image, cv2.cvtColor(template, cv2.COLOR_BGR2LUV), cv2.TM_SQDIFF)
    result = match.min() < 200000#500000 and len([v for v in match.reshape(-1) if v < 2*match.min()]) < 70)
    print(match.min(), match.max(), round(math.log(match.min(), 2)), round(math.log(match.max(), 2)),
          len([v for v in match.reshape(-1) if v < 2*match.min()]), result)
    match = np.clip(match, 0, 2**18)
    match = np.broadcast_to(match / match.max(), (match.shape[0], n))
    return result
[Counter(map(fn, images)) for images in (negatives, positives)]

664952.44 1954082.8 19 21 38 False
1143504.5 1954082.8 20 21 98 False
676506.25 1903990.6 19 21 39 False
674853.7 1903990.6 19 21 39 False
690872.8 1916141.1 19 21 40 False
690811.2 1905965.4 19 21 40 False
690811.2 1905965.4 19 21 40 False
672233.6 1911845.5 19 21 39 False
700966.75 1927937.1 19 21 44 False
717655.25 1916532.4 19 21 47 False
1202959.8 1912218.5 20 21 98 False
1198605.6 1912730.6 20 21 98 False
1201127.6 1940632.9 20 21 98 False
1187724.1 1945740.6 20 21 98 False
1188446.6 1945068.6 20 21 98 False
1151839.6 1903263.6 20 21 98 False
1151887.5 1901828.9 20 21 98 False
1152462.1 1903263.6 20 21 98 False
1157052.1 1902618.1 20 21 98 False
1173524.8 1918533.1 20 21 98 False
1173924.4 1918509.8 20 21 98 False
1147879.2 1915736.5 20 21 98 False
1162822.1 1925088.6 20 21 98 False
1162118.1 1925120.2 20 21 98 False
1131461.4 1909209.2 20 21 98 False
1158085.9 1908942.6 20 21 98 False
1163317.0 1915416.2 20 21 98 False
1164378.0 1915586.0 20 21 98 False
1164795.1 1915586.0 20 21

68831.3 87977.09 16 16 98 True
57323.02 87260.62 16 16 98 True
59632.062 89719.01 16 16 98 True
61902.79 96080.71 16 17 98 True
59478.06 104029.41 16 17 98 True
59657.082 97607.92 16 17 98 True
495758.44 1609699.4 19 21 71 False
506012.28 1610009.9 19 21 71 False
420474.88 1738809.5 19 21 31 False
561223.6 1780663.4 19 21 80 False
461902.4 1721700.9 19 21 58 False
639297.5 1397714.9 19 20 92 False
640552.5 1397903.2 19 20 92 False
641885.3 1395637.5 19 20 93 False
615935.56 1345026.6 19 20 93 False
608287.44 1301549.2 19 20 93 False
635248.5 1339468.1 19 20 94 False
630875.1 1340425.4 19 20 94 False
634953.6 1342089.1 19 20 94 False
57853.65 101399.914 16 17 98 True
68942.13 92086.695 16 16 98 True
69064.26 90142.44 16 16 98 True
69581.22 87391.164 16 16 98 True
65455.574 265138.34 16 18 92 True
64435.383 285686.2 16 18 90 True
54310.746 347075.28 16 18 89 True
52708.883 322081.12 16 18 89 True
53303.98 324553.72 16 18 89 True
53303.98 324553.72 16 18 89 True
54960.17 321383.12 16 18 9

80892.84 1782292.4 16 21 26 True
80892.84 1782292.4 16 21 26 True
80892.84 1782292.4 16 21 26 True
80892.84 1771273.4 16 21 26 True
80892.84 1770242.6 16 21 26 True
80892.84 1770242.6 16 21 26 True
80892.84 1770242.6 16 21 26 True
80892.84 1771086.8 16 21 26 True
80892.84 1771087.9 16 21 26 True
80892.84 1771087.9 16 21 26 True
80925.85 1771087.9 16 21 23 True
80877.52 1771087.9 16 21 25 True
80877.52 1771087.9 16 21 25 True
80877.52 1771087.9 16 21 25 True
80877.52 1771087.9 16 21 25 True
80877.52 1771087.9 16 21 25 True
80877.52 1771087.9 16 21 25 True
80877.52 1771087.9 16 21 25 True
80877.52 1771087.9 16 21 25 True
80877.52 1771087.9 16 21 25 True
79551.41 1743566.2 16 21 25 True
80514.01 1745786.9 16 21 26 True
79669.56 1745776.1 16 21 26 True
80449.914 1745776.1 16 21 27 True
81457.875 1745776.1 16 21 29 True
81457.875 1745776.1 16 21 27 True
82696.09 1745776.1 16 21 34 True
82959.08 1745776.1 16 21 35 True
82576.57 1761444.6 16 21 33 True
82942.37 1761444.6 16 21 35 True
81531.6

72828.96 1697761.4 16 21 4 True
71984.53 1690024.2 16 21 4 True
72911.95 1690024.2 16 21 4 True
71984.53 1665133.0 16 21 4 True
73537.7 1661203.1 16 21 4 True
71991.8 1660507.9 16 21 4 True
71561.35 1660507.9 16 21 4 True
73537.7 1660507.9 16 21 4 True
71991.8 1680098.0 16 21 4 True
72919.73 1820611.8 16 21 4 True
71991.8 1840203.2 16 21 4 True
72919.73 1859031.0 16 21 4 True
71991.8 1851481.5 16 21 4 True
72919.73 1865892.1 16 21 4 True
89660.46 1845012.4 16 21 5 True
71991.8 1882838.0 16 21 4 True
71561.35 1870708.9 16 21 4 True
71561.35 1854002.9 16 21 4 True
71561.35 1840543.1 16 21 4 True
71561.35 1811098.9 16 21 4 True
71561.35 1815730.4 16 21 4 True
174865.33 2726194.2 17 21 5 True
173563.77 2749598.0 17 21 5 True
16380.513 641582.44 14 19 4 True
13818.563 641206.44 14 19 4 True
13332.525 633090.4 14 19 4 True
15142.361 631400.44 14 19 4 True
13325.201 611840.44 14 19 4 True
15401.491 598640.6 14 19 4 True
14064.498 575347.5 14 19 4 True
7980.327 491924.84 13 19 3 True
16562.746

[Counter({False: 392, True: 58}), Counter({True: 450})]

In [13]:
images = list(pickle_iter(r"D:\Valorant\Killed\Color\00039.pickle"))
len(images)

8000

In [14]:
positives = (images[3700:5565], images[5645:5734], images[5783:5849],
             images[7033:7545], images[7548:7555], images[7582:7602])
[(len(l), list(prime_factors(len(l)))) for l in positives]

[(1865, [5, 373]),
 (89, [89]),
 (66, [2, 3, 11]),
 (512, [2, 2, 2, 2, 2, 2, 2, 2, 2]),
 (7, [7]),
 (20, [2, 2, 5])]

In [15]:
def fn(images):
    images = list(it.chain.from_iterable(images))
    g = grouper(images, math.ceil(math.sqrt(len(images))), fillvalue=np.zeros_like(images[0]))
    g = (np.hstack([cv2.resize(image, (20, 20)) for image in g]) for g in g)
    show_and_wait(np.vstack(list(g)))
    cv2.destroyAllWindows()
    return list(prime_factors(len(images)))
fn(positives)

[3, 853]

In [16]:
show_and_wait(np.hstack([np.vstack(t[:7]) for t in positives]))
cv2.destroyAllWindows()

In [17]:
extents = [
    (49, left, 49 + height, right),
    (73, left, 73 + height, right),
    (73, left, 73 + height, right),
    (60, left, 60 + height, right),
    (60, left, 60 + height, right),
    (60, left, 60 + height, right),
]
g = zip(positives, extents)
g = (zip(a, it.repeat(b)) for a, b in g)
g = it.chain.from_iterable(g)
g = (i[top:bottom, left:right] for i, (top, left, bottom, right) in g)
color = np.stack(list(g)).mean(axis=(0, 1, 2))
show_and_wait(np.broadcast_to(color, (222, 222, 3)))
cv2.destroyAllWindows()
color.shape, color

((3,), array([0.5217048 , 0.66696596, 0.69452584], dtype=float32))

In [18]:
color = np.array([0.5691015, 0.6987058, 0.7345293], dtype=np.float32)

In [19]:
template = np.broadcast_to(color, (height, width, 3))
match = cv2.matchTemplate(positives[0][0][:, left:right], template, cv2.TM_SQDIFF)
show_and_wait(np.broadcast_to(match / match.max(), (match.shape[0], 222)))
cv2.destroyAllWindows()
match.reshape(-1), match.min(), match.max()

(array([209.52858  , 209.74431  , 210.21208  , 210.67996  , 211.14767  ,
        211.77586  , 212.4361   , 213.07112  , 213.99413  , 214.89342  ,
        215.79277  , 216.75017  , 217.75876  , 218.69626  , 219.68343  ,
        220.67067  , 221.49747  , 222.29224  , 223.08711  , 223.59387  ,
        224.10077  , 224.60759  , 225.01674  , 225.0664   , 225.11601  ,
        225.11601  , 225.0664   , 225.07426  , 225.01169  , 224.85088  ,
        224.8095   , 224.75163  , 224.69385  , 224.63605  , 224.49933  ,
        224.36263  , 220.46913  , 208.77858  , 188.468    , 168.13968  ,
        147.97995  , 127.67159  , 107.40695  ,  87.151054 ,  66.87335  ,
         46.674675 ,  26.508512 ,  10.162605 ,   1.6513363,   1.6070504,
          1.570143 ,  14.353399 ,  32.48523  ,  50.209373 ,  67.25989  ,
         84.44744  , 102.24433  , 120.94004  , 140.18889  , 160.07068  ,
        180.26868  , 200.88287  , 208.88553  , 211.70177  , 214.73521  ,
        218.74294  , 221.27888  , 223.0455   , 224.

In [20]:
negatives = ((0, 3700), (5565, 5645), (5734, 5783), (5849, 7033),
             (7545, 7548), (7555, 7582), (7602, 8000))
negatives = list(it.chain.from_iterable(map(list, it.starmap(range, negatives))))
negatives = sorted(random.sample(negatives, sum(len(l) for l in positives)))
print(list(prime_factors(len(negatives))))
negatives = [images[i] for i in negatives]
show_square(negatives, size=20)
cv2.destroyAllWindows()

[3, 853]


In [21]:
def fn(image):
    match = cv2.matchTemplate(image[:, left:right], template, cv2.TM_SQDIFF)
    return match
image = np.hstack(list(map(fn, negatives + list(it.chain.from_iterable(positives)))))
show_and_wait(image / image.max())
cv2.destroyAllWindows()

In [22]:
def fn(image):
    n, d = 25, math.log(2)
    image = cv2.cvtColor(image[:, left:right], cv2.COLOR_BGR2LUV)
    match = cv2.matchTemplate(image, cv2.cvtColor(template, cv2.COLOR_BGR2LUV), cv2.TM_SQDIFF)
    result = match.min() < 200000#500000 and len([v for v in match.reshape(-1) if v < 2*match.min()]) < 70)
    print(match.min(), match.max(), round(math.log(match.min(), 2)), round(math.log(match.max(), 2)),
          len([v for v in match.reshape(-1) if v < 2*match.min()]), result)
    match = np.clip(match, 0, 2**18)
    match = np.broadcast_to(match / match.max(), (match.shape[0], n))
    return result
[Counter(map(fn, images)) for images in (negatives, list(it.chain.from_iterable(positives)))]

1222817.5 1343077.0 20 20 98 False
1334626.2 1445797.0 20 20 98 False
1421206.4 1530986.2 20 21 98 False
1452475.0 1556084.5 20 21 98 False
1421958.1 1569271.1 20 21 98 False
1421504.2 1562586.1 20 21 98 False
1423788.1 1523091.0 20 21 98 False
1403269.5 1494479.0 20 21 98 False
1378629.1 1447466.8 20 20 98 False
1019819.06 1102775.5 20 20 98 False
776404.1 835961.9 20 20 98 False
277851.8 361011.94 18 18 98 False
295751.3 365895.7 18 18 98 False
272952.6 335280.38 18 18 98 False
520749.34 556117.75 19 19 98 False
783129.44 835577.2 20 20 98 False
897323.3 942916.9 20 20 98 False
892843.1 936451.0 20 20 98 False
883962.0 926273.8 20 20 98 False
894395.9 918061.3 20 20 98 False
891522.94 921465.9 20 20 98 False
847976.2 907284.1 20 20 98 False
858383.8 904893.7 20 20 98 False
854499.8 916641.06 20 20 98 False
855783.75 898932.75 20 20 98 False
796826.44 890581.5 20 20 98 False
747168.7 890446.94 20 20 98 False
736175.94 887575.94 19 20 98 False
757489.56 921813.0 20 20 98 False
782872.8

950619.4 1370849.1 20 20 98 False
878844.25 1002507.7 20 20 98 False
863521.75 996625.5 20 20 98 False
824052.44 944005.75 20 20 98 False
820669.1 982045.8 20 20 98 False
1023167.4 1337748.2 20 20 98 False
938559.2 1167651.0 20 20 98 False
813033.5 896128.06 20 20 98 False
793766.25 920821.6 20 20 98 False
871415.9 1136473.6 20 20 98 False
831001.6 1553202.2 20 21 98 False
936475.2 1518244.5 20 21 98 False
934545.4 1247810.5 20 20 98 False
882564.06 1237815.0 20 20 98 False
926803.6 1411386.9 20 20 98 False
930006.25 1514133.5 20 21 98 False
923252.7 1562193.1 20 21 98 False
929263.8 1510930.0 20 21 98 False
927287.3 1410610.4 20 20 98 False
929273.9 1515361.0 20 21 98 False
910788.25 1595371.1 20 21 98 False
926925.7 1588385.1 20 21 98 False
927540.2 1579039.9 20 21 98 False
933209.2 1543843.0 20 21 98 False
933721.3 1561334.1 20 21 98 False
918659.7 1593249.6 20 21 98 False
896836.25 1599747.0 20 21 98 False
877275.3 1607971.9 20 21 98 False
870078.9 1617275.8 20 21 98 False
931663.4

1601061.9 1705041.9 21 21 98 False
1687526.4 1974519.6 21 21 98 False
1680089.4 1962902.0 21 21 98 False
1673047.6 1962480.2 21 21 98 False
1672105.1 1971853.9 21 21 98 False
1654659.8 1961528.8 21 21 98 False
1601533.2 1922762.0 21 21 98 False
1588162.0 1928829.5 21 21 98 False
1536699.4 1916890.1 21 21 98 False
1542438.6 1844366.9 21 21 98 False
1493647.5 1835013.6 21 21 98 False
1489295.4 1825931.0 21 21 98 False
1489057.9 1819341.6 21 21 98 False
1484397.2 1792918.9 21 21 98 False
1479370.6 1805977.8 20 21 98 False
1483050.9 1801556.0 21 21 98 False
1373413.1 1714675.4 20 21 98 False
1119742.1 1658308.0 20 21 98 False
1112818.0 1552786.1 20 21 98 False
1648560.9 1713106.5 21 21 98 False
1631618.8 1675387.9 21 21 98 False
1623020.4 1671811.8 21 21 98 False
1661794.8 1906543.5 21 21 98 False
1658379.9 1888990.2 21 21 98 False
1656149.5 1892939.4 21 21 98 False
1649573.1 1862547.4 21 21 98 False
1630389.6 1786869.0 21 21 98 False
1595430.0 1709947.6 21 21 98 False
1583316.9 1767196.8 

1129157.6 2839654.8 20 21 49 False
1132622.6 2839654.8 20 21 49 False
1132455.5 2839654.8 20 21 49 False
1129442.8 2839654.8 20 21 49 False
1122837.2 2843416.8 20 21 49 False
1121749.6 2843494.0 20 21 49 False
1113623.9 2843427.5 20 21 48 False
1108770.4 2843554.2 20 21 48 False
1106179.8 2843554.2 20 21 48 False
1089409.0 2849814.8 20 21 47 False
1088064.5 2849814.8 20 21 47 False
1086180.4 2850060.0 20 21 47 False
1087847.0 2849814.8 20 21 47 False
1087342.1 2849814.8 20 21 47 False
1096115.4 2849814.8 20 21 47 False
1108109.9 2849814.8 20 21 48 False
1114226.9 2849814.8 20 21 48 False
1125171.6 2843595.5 20 21 49 False
1137916.5 2842171.2 20 21 49 False
1138084.8 2841604.2 20 21 49 False
1142701.4 2841604.2 20 21 49 False
1142631.6 2841604.2 20 21 49 False
1143913.6 2841604.2 20 21 49 False
1140742.8 2841604.2 20 21 49 False
1148470.6 2840703.5 20 21 50 False
1152371.2 2840788.0 20 21 50 False
1158443.1 2841604.2 20 21 50 False
1165721.6 2841947.8 20 21 50 False
1180162.9 2842141.2 

1242855.2 1318840.5 20 20 98 False
1255459.8 1654353.4 20 21 98 False
1261956.1 1819045.5 20 21 98 False
1264678.1 1785715.1 20 21 98 False
1268397.4 1597166.4 20 21 98 False
1278601.6 1314658.8 20 20 98 False
1274438.4 1306473.0 20 20 98 False
1272029.6 1311180.1 20 20 98 False
1299986.1 1420721.1 20 20 98 False
1395367.1 1521005.2 20 21 98 False
1477968.0 1601253.2 20 21 98 False
1583600.0 1667391.2 21 21 98 False
1587062.2 1631607.5 21 21 98 False
1560835.4 1596691.8 21 21 98 False
1517854.9 1556718.0 21 21 98 False
1507494.2 1547541.9 21 21 98 False
1486554.1 1530182.0 21 21 98 False
1496196.1 1531630.9 21 21 98 False
1509921.4 1531999.8 21 21 98 False
1498056.2 1530015.4 21 21 98 False
1426649.1 1478558.9 20 20 98 False
1332977.1 1386289.6 20 20 98 False
1161737.9 1210016.6 20 20 98 False
1151403.5 1190956.9 20 20 98 False
1135284.1 1204089.9 20 20 98 False
1311061.1 1617201.0 20 21 98 False
1253891.5 1505372.9 20 21 98 False
1218209.8 1407637.0 20 20 98 False
1213387.2 1371411.6 

...

29193.521 1336041.4 15 20 3 True
29557.314 1342398.2 15 20 3 True
29597.549 1354316.0 15 20 3 True
29557.314 1353937.1 15 20 3 True
29723.744 1355373.5 15 20 3 True
29671.695 1351954.4 15 20 3 True
29359.08 1350194.1 15 20 3 True
29359.08 1344651.5 15 20 3 True
29335.34 1352925.8 15 20 3 True
29048.934 1351372.2 15 20 3 True
28948.363 1359714.2 15 20 3 True
29065.531 1346077.5 15 20 3 True
29939.764 1352092.4 15 20 3 True
30995.887 1342290.1 15 20 3 True
30995.887 1343457.4 15 20 3 True
30995.887 1338783.2 15 20 3 True
30995.887 1338603.4 15 20 3 True
30995.887 1336691.6 15 20 3 True
111894.01 2224083.2 17 21 5 True
209764.7 2299063.2 18 21 6 False
182186.48 2314564.5 17 21 6 True
158351.86 2360362.2 17 21 5 True
122435.5 2755538.5 17 21 5 True
130699.11 2825291.0 17 21 5 True
119756.79 2533778.8 17 21 5 True
178533.45 1957745.2 17 21 6 True
298020.25 1757859.8 18 21 10 False
358787.8 2061391.0 18 21 12 False
358119.53 2039952.4 18 21 12 False
311556.72 1805402.2 18 21 10 False
251663.

114356.68 2780642.8 17 21 5 True
98975.87 2906988.8 17 21 5 True
98051.625 2910729.2 17 21 5 True
43622.82 2717138.2 15 21 4 True
37240.207 1476813.4 15 20 3 True
34109.996 1422164.1 15 20 3 True
28546.355 1402075.4 15 20 3 True
28500.633 1432044.5 15 20 3 True
25991.193 1470926.5 15 20 3 True
26698.06 1467138.4 15 20 3 True
24448.748 1376367.9 15 20 3 True
28555.37 1362630.8 15 20 3 True
39792.055 1597537.5 15 21 3 True
47837.97 1852613.5 16 21 3 True
37742.85 1333390.0 15 20 3 True
37742.85 1333390.0 15 20 3 True
57670.445 1807392.0 16 21 4 True
39430.016 1370230.9 15 20 3 True
36559.645 1414647.2 15 20 3 True
28333.848 1335831.5 15 20 4 True
27743.45 1334974.5 15 20 3 True
39667.45 1568293.4 15 21 3 True
37642.473 1447214.0 15 20 3 True
32313.953 1607615.8 15 21 3 True
36022.01 1591943.9 15 21 3 True
30353.98 1327868.8 15 20 3 True
31445.363 1326447.9 15 20 3 True
28951.828 1326447.9 15 20 4 True
39651.6 1326447.9 15 20 5 True
49214.703 1326447.9 16 20 5 True
49298.895 1326447.9 16 

74385.555 1824815.8 16 21 9 True
116121.02 1845660.6 17 21 17 True
142307.97 1853137.6 17 21 14 True
64578.67 1881582.9 16 21 30 True
64746.418 1846180.1 16 21 33 True
73290.42 1848011.8 16 21 58 True
103344.95 1855876.2 17 21 64 True
93379.586 1839970.0 17 21 53 True
57042.066 1852960.6 16 21 12 True
97157.35 1835802.2 17 21 30 True
57801.34 1839346.1 16 21 9 True
84127.24 1842309.0 16 21 5 True
77587.18 1848053.1 16 21 5 True
79263.83 1852121.9 16 21 5 True
78378.19 1856289.5 16 21 5 True
78885.02 1857912.0 16 21 5 True
78998.2 1859629.4 16 21 5 True
65362.88 1859629.4 16 21 5 True
73795.28 1859629.4 16 21 5 True
73867.234 1865237.4 16 21 5 True
76010.586 1865461.6 16 21 5 True
77234.56 1863482.8 16 21 5 True
79659.05 1864089.9 16 21 5 True
85249.09 1935058.8 16 21 5 True
85302.72 1951772.2 16 21 5 True
98785.78 2010292.1 17 21 5 True
101801.016 2122359.5 17 21 5 True
109988.13 2255829.0 17 21 5 True
116147.984 2275798.5 17 21 5 True
132649.02 2371375.5 17 21 5 True
133331.95 2360413

51344.113 1916099.1 16 21 37 True
46185.312 1916602.8 15 21 35 True
49557.29 1917821.2 16 21 36 True
47244.258 1903787.5 16 21 35 True
54268.133 1905296.6 16 21 37 True
49963.266 1897183.9 16 21 36 True
55373.41 1897254.8 16 21 37 True
55107.613 1897398.8 16 21 37 True
56315.8 1895353.0 16 21 38 True
55817.305 1894825.8 16 21 38 True
54087.305 1893635.0 16 21 37 True
56604.918 1892822.6 16 21 38 True
53795.902 1914878.0 16 21 36 True
53093.047 1916252.2 16 21 36 True
54556.426 1916252.2 16 21 37 True
60000.45 1916252.2 16 21 38 True
56688.67 1916047.9 16 21 38 True
59920.44 1905193.1 16 21 38 True
59196.92 1905120.4 16 21 38 True
58658.777 1905256.9 16 21 38 True
193240.45 1904701.1 18 21 15 True
193680.28 1904701.1 18 21 15 True
194293.75 1904701.1 18 21 15 True
194331.56 1904701.1 18 21 15 True
194258.12 1904701.1 18 21 15 True
194258.12 1904701.1 18 21 15 True
194258.12 1904701.1 18 21 15 True
194258.12 1904701.1 18 21 15 True
206810.3 1916487.8 18 21 15 False
206810.3 1916487.8 18 

[Counter({False: 2403, True: 156}), Counter({True: 2341, False: 218})]